In [ ]:
client.

In [1]:
from __future__ import print_function
import aerospike
import logging
import time

logging.getLogger().setLevel(logging.WARNING)

maxLoops = 100

aeroNameSpace = "serving_layer"
aeroSet = "customers"

In [2]:
# Configure the client
config = {
    'hosts': [ ('127.0.0.1', 3000) ]
}

In [3]:
# Create a client and connect it to the cluster
try:
    client = aerospike.client(config).connect()
except:
    import sys
    print("failed to connect to the cluster with", config['hosts'])
    sys.exit(1)

In [ ]:
def add_customer(customer_id, phone_number, lifetime_value):
    try:
        client.put( (aeroNameSpace, aeroSet, customer_id), {
            'phone_number': phone_number,
            'lifetime_value': lifetime_value
         })
    except Exception as e:
        logger.error("error: {0}".format(e))

In [ ]:
def get_ltv_by_id(customer_id):
    try:
        (key, metadata, record) = client.get(customer_id)
        return record.get('ltv')
    except Exception as e:
        logger.error('Requested non-existent customer ' + str(customer_id))

In [ ]:
def get_ltv_by_phone(phone_number):
    # Здесь, вероятно, потребуется как-то искать по доп. индексу по телефону
    # Либо сделать ход конем, что, по-моему, даже интереснее,
    #   и создать отдельный сет (таблицу) для связи телефон--кастомер_ид, где ключом выступает уже телефон
    #   или для этих целей в аэроспайке есть какие-то проекции... :)
    try:
        (key, metadata, record) = client.get(customer_id)
        return record.get('ltv')
    except Exception as e:
        logging.error('Requested phone number is not found ' + str(phone_number))

In [ ]:
startTime = time.time()
for i in range(0, maxLoops):
    add_customer(i,i,i + 1)
print("Insert time by id: ", (time.time()-startTime))

startTime = time.time()
for i in range(0, maxLoops):
    assert (i + 1 == get_ltv_by_id(i)), "No LTV by ID " + str(i)
print("Select time by id: ", (time.time()-startTime))

startTime = time.time()
for i in range(0, maxLoops):
    assert (i + 1 == get_ltv_by_phone(i)), "No LTV by phone " + str(i)
print("Select time by phone: ", (time.time()-startTime))

print ("All is OK! :)")

In [29]:
# Close the connection to the Aerospike cluster
client.close()